<a href="https://colab.research.google.com/github/bhavinithak/LLM/blob/main/OpenAI_LLM_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tiktoken

In [ ]:
!pip install openai

In [3]:
from tiktoken import encoding_for_model

In [5]:
enc=encoding_for_model("text-davinci-003")
toks=enc.encode("They are playing")
toks

[2990, 389, 2712]

In [6]:
[enc.decode_single_token_bytes(o).decode('utf-8') for o in toks]

['They', ' are', ' playing']

In [33]:
import openai

In [34]:
canadian_sys="You are a Canadian LLM that uses Canadian slang and analogies whenever possible"

In [35]:
client = OpenAI(
    api_key=""
)

In [36]:
c=client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": canadian_sys},
              {"role": "user", "content": "What is money?"}]
)

In [40]:
print(c.choices[0].message.content)

Money, eh? It's like maple syrup for the economy - necessary for things to flow smoothly, but too much of it in one place can make things sticky. Money is basically a medium of exchange used to buy goods and services, so you can think of it as the loonie in your wallet that lets you grab a double-double at Timmy's. Just remember, it's not about how much money you have, but how you use it, like trying to hit a corner in a hockey game - it's all about strategy and finesse.


In [41]:
print(c.usage)

CompletionUsage(completion_tokens=112, prompt_tokens=30, total_tokens=142)


In [42]:
def askgpt(user, system=None, model="gpt-3.5-turbo", **kwargs):
    msgs = []
    if system: msgs.append({"role": "system", "content": system})
    msgs.append({"role": "user", "content": user})
    return client.chat.completions.create(model=model, messages=msgs, **kwargs)

In [48]:
a1=askgpt('What is the meaning of life?',system=canadian_sys)
a1.choices[0].message.content

"Oh, that's a deep question, eh? The meaning of life is like a double-double from Tim Hortons - everyone's got their own unique blend, and you gotta figure out what tastes right for you. Some folks find meaning in family, others in their work, and some in the pursuit of happiness. It's all about finding your own recipe for a fulfilling existence, ya know?"

# Code Interpreter

In [49]:
from pydantic import create_model
import inspect, json
from inspect import Parameter

In [66]:
import tokenize, ast
from io import BytesIO

In [50]:
def sums(a:int, b:int=1):
    "Adds a + b"
    return a + b

In [51]:
def schema(f):
    kw = {n:(o.annotation, ... if o.default==Parameter.empty else o.default)
          for n,o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [52]:
schema(sums)

{'name': 'sums',
 'description': 'Adds a + b',
 'parameters': {'properties': {'a': {'title': 'A', 'type': 'integer'},
   'b': {'default': 1, 'title': 'B', 'type': 'integer'}},
  'required': ['a'],
  'title': 'Input for `sums`',
  'type': 'object'}}

In [53]:
c = askgpt("Use the `sum` function to solve this: What is 6+3?",
           system = "You must use the `sum` function instead of adding yourself.",
           functions=[schema(sums)])

In [54]:
m = c.choices[0].message
m

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"a":6,"b":3}', name='sums'), tool_calls=None)

In [55]:
k = m.function_call.arguments
print(k)

{"a":6,"b":3}


In [56]:

funcs_ok = {'sums', 'python'}

In [57]:
def call_func(c):
    fc = c.choices[0].message.function_call
    if fc.name not in funcs_ok: return print(f'Not allowed: {fc.name}')
    f = globals()[fc.name]
    return f(**json.loads(fc.arguments))

In [58]:
call_func(c)

9

In [67]:
def run(code):
    tree = ast.parse(code)
    last_node = tree.body[-1] if tree.body else None

    # If the last node is an expression, modify the AST to capture the result
    if isinstance(last_node, ast.Expr):
        tgts = [ast.Name(id='_result', ctx=ast.Store())]
        assign = ast.Assign(targets=tgts, value=last_node.value)
        tree.body[-1] = ast.fix_missing_locations(assign)

    ns = {}
    exec(compile(tree, filename='<ast>', mode='exec'), ns)
    return ns.get('_result', None)

In [68]:
def python(code:str):
    "Return result of executing `code` using python. If execution not permitted, returns `#FAIL#`"
    go = input(f'Proceed with execution?\n```\n{code}\n```\n')
    if go.lower()!='y': return '#FAIL#'
    return run(code)

In [69]:
c = askgpt("What is 12 factorial?",
           system = "Use python for any required computations.",
           functions=[schema(python)])

In [70]:
call_func(c)

Proceed with execution?
```
import math
math.factorial(12)
```
y


479001600

In [71]:
c = client.chat.completions.create(
    model="gpt-3.5-turbo",
    functions=[schema(python)],
    messages=[{"role": "user", "content": "What is 12 factorial?"},
              {"role": "function", "name": "python", "content": "479001600"}])

In [73]:
c.choices[0].message.content

'12 factorial is 479,001,600.'

In [74]:
c = askgpt("What is the capital of France?",
           system = "Use python for any required computations.",
           functions=[schema(python)])

In [75]:
c.choices[0].message.content

'The capital of France is Paris.'